In [1]:
!pip install opencv-python torchsummary scikit-learn torchviz utils

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
import utils
from torch.nn.functional import one_hot
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import SubsetRandomSampler

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.get_arch_list(), device)

['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86'] cuda


In [4]:
from __future__ import print_function
import argparse
import datetime
import matplotlib.pyplot as plt
from IPython.display import HTML
from IPython.display import clear_output
from tqdm import tqdm
import os
from tqdm import notebook
from sklearn.metrics import classification_report, confusion_matrix
import gc
import matplotlib.colors as mat_color
import numpy as np
from PIL import Image
import random
import cv2
from torchvision.datasets import ImageNet, ImageFolder
from torch.utils.data import DataLoader
from torchsummary import summary
from torchviz import make_dot

In [5]:
def initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()

class Generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, input_dim=100, output_dim=1, input_size=32, class_num=10):
        super(Generator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.BatchNorm1d(128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.ReLU(),
        )
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.output_dim, 4, 2, 1),
            nn.Tanh(),
        )
        initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        x = x.view(-1, 128, (self.input_size // 4), (self.input_size // 4))
        x = self.deconv(x)

        return x

    
class Discriminator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    def __init__(self, input_dim=1, output_dim=1, input_size=32, class_num=10):
        super(Discriminator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.conv = nn.Sequential(
            nn.Conv2d(self.input_dim, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(128 * (self.input_size // 4) * (self.input_size // 4), 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
        )
        self.dc = nn.Sequential(
            nn.Linear(1024, self.output_dim),
            nn.Sigmoid(),
        )
        self.cl = nn.Sequential(
            nn.Linear(1024, self.class_num),
        )
        initialize_weights(self)

    def forward(self, input):
        x = self.conv(input)
        x = x.view(-1, 128 * (self.input_size // 4) * (self.input_size // 4))
        x = self.fc1(x)
        d = self.dc(x)
        c = self.cl(x)

        return d, c


In [6]:
base_path = './data'
base_folder = "Covid-19 Image Dataset"
classic_folder = 'Coivd-19_Classic'
synthetic_folder = 'Coivd-19_Synthetic'
data_dir = os.path.join(base_path, classic_folder)

In [7]:
batch_size = 32
train_path = os.path.join(data_dir, "train")
test_path = os.path.join(data_dir, "test")

In [8]:
labels = os.listdir(train_path)
print(labels)
no_norm = mat_color.Normalize(vmin=0, vmax=255, clip=False)
label_dict = {
    0 : 'Covid', 
    1 : 'Normal', 
    2 : 'Viral Pneumonia'
}

['Covid', 'Normal', 'Viral Pneumonia']


In [9]:
# Number of training epochs
num_epochs = 200
#All images will be resized to this size using a transformer.
img_size = 128 * 2
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = img_size
# Size of feature maps in generator
ngf = img_size
# Size of feature maps in discriminator
ndf = img_size
# No of labels
nb_label = len(labels)
# Learning rate for optimizers
lr = 0.00000010
lr_d = 0.00000002
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Beta2 hyperparam for Adam optimizers
beta2 = 0.999

real_label = 1.
fake_label = 0.
s_criterion = nn.BCELoss().to(device) #For synthesizing
c_criterion = nn.CrossEntropyLoss().to(device) #For classification

s_label = torch.FloatTensor(batch_size).to(device)
c_label = torch.FloatTensor(batch_size, nb_label).to(device)
noise = torch.FloatTensor(batch_size, nz).to(device)

s_label = Variable(s_label)
c_label = Variable(c_label)
print(s_label.shape)
print(c_label.shape)
print(noise.shape)

noise = Variable(noise)

noise_fixed = torch.FloatTensor(3, nz).to(device)
noise_data = np.random.normal(0, 1, (3, nz))
noise_fixed.data.copy_(torch.tensor(noise_data))
noise_fixed = Variable(noise_fixed)

label_fixed = torch.FloatTensor(3, 3).to(device)
label_onehot = np.zeros((3, nb_label))
label_fixed_data = [0, 1, 2]
label_onehot[np.arange(1), label_fixed_data] = 1
label_fixed.data.copy_(torch.tensor(label_onehot))
label_fixed = Variable(label_fixed)

print(noise_fixed.shape)
print(label_fixed.shape)

torch.Size([32])
torch.Size([32, 3])
torch.Size([32, 256])
torch.Size([3, 256])
torch.Size([3, 3])


In [10]:
for func in [
    lambda: os.mkdir(os.path.join('.', 'GANAug')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/model')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/plots')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/model/ACGAN')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/plots/ACGAN')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/output_images')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/output_images/ACGAN')),
    lambda: os.mkdir(os.path.join('.', 'GANAug/output_images/ACGAN/' + label_dict[0])),
    lambda: os.mkdir(os.path.join('.', 'GANAug/output_images/ACGAN/' + label_dict[1])),
    lambda: os.mkdir(os.path.join('.', 'GANAug/output_images/ACGAN/' + label_dict[2]))]:  # create directories
    try:
        func()
    except Exception as error:
        print(error)
        continue

[Errno 17] File exists: './GANAug'
[Errno 17] File exists: './GANAug/model'
[Errno 17] File exists: './GANAug/plots'
[Errno 17] File exists: './GANAug/model/ACGAN'
[Errno 17] File exists: './GANAug/plots/ACGAN'
[Errno 17] File exists: './GANAug/output_images'
[Errno 17] File exists: './GANAug/output_images/ACGAN'
[Errno 17] File exists: './GANAug/output_images/ACGAN/Covid'
[Errno 17] File exists: './GANAug/output_images/ACGAN/Normal'
[Errno 17] File exists: './GANAug/output_images/ACGAN/Viral Pneumonia'


In [11]:
matrix_fields = [
    'G_losses',
    'G_class_losses',
    'G_syn_losses',
    'D_losses',
    'D_real_losses',
    'D_fake_losses',
    'D_class_losses',
    'D_syn_losses',
    'Accuracy',
    'Losses'
]
metrics = {field: list() for field in matrix_fields}

In [12]:
def load_dataset(train_dir=train_path, test_dir=test_path):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=1)

    test_data = datasets.ImageFolder(test_dir ,transform=transform)
    test_loader = DataLoader(test_data, batch_size, shuffle=True, num_workers=1)

    return train_loader, test_loader, train_data, test_data

In [13]:
train_loader, test_loader, train_data, test_data = load_dataset()

In [14]:
generator = Generator(input_dim=nz, output_dim=nc, input_size=ngf, class_num=nb_label).to(device)
discriminator = Discriminator(input_dim=nc, output_dim=1, input_size=ndf, class_num=nb_label).to(device)

# setup optimizer
optimizerD = optim.Adam(discriminator.parameters(), lr=lr_d, betas=(beta1, beta2))
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))

In [15]:
g_output = generator(torch.rand((batch_size, nz)).to(device), torch.ones((batch_size, nb_label)).to(device))
print(g_output.shape)

torch.Size([32, 3, 256, 256])


In [16]:
s_output, c_output = discriminator(torch.rand(g_output.shape).to(device))
print(s_output.shape)
print(c_output.shape)
del g_output, s_output, c_output

torch.Size([32, 1])
torch.Size([32, 3])


In [17]:
!pip install torchviz

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [18]:
print(generator)
print(discriminator)

Generator(
  (fc): Sequential(
    (0): Linear(in_features=259, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=524288, bias=True)
    (4): BatchNorm1d(524288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (deconv): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): Tanh()
  )
)
Discriminator(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_runni

In [19]:
SHOW = True
if SHOW:
    summary(generator, [[nz], [nc]], batch_size=16, device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [16, 1024]         266,240
       BatchNorm1d-2                 [16, 1024]           2,048
              ReLU-3                 [16, 1024]               0
            Linear-4               [16, 524288]     537,395,200
       BatchNorm1d-5               [16, 524288]       1,048,576
              ReLU-6               [16, 524288]               0
   ConvTranspose2d-7         [16, 64, 128, 128]         131,136
       BatchNorm2d-8         [16, 64, 128, 128]             128
              ReLU-9         [16, 64, 128, 128]               0
  ConvTranspose2d-10          [16, 3, 256, 256]           3,075
             Tanh-11          [16, 3, 256, 256]               0
Total params: 538,846,403
Trainable params: 538,846,403
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05


In [20]:
if SHOW:
    summary(discriminator, (nc, img_size, img_size), batch_size=16, device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 64, 128, 128]           3,136
         LeakyReLU-2         [16, 64, 128, 128]               0
            Conv2d-3          [16, 128, 64, 64]         131,200
       BatchNorm2d-4          [16, 128, 64, 64]             256
         LeakyReLU-5          [16, 128, 64, 64]               0
            Linear-6                 [16, 1024]     536,871,936
       BatchNorm1d-7                 [16, 1024]           2,048
         LeakyReLU-8                 [16, 1024]               0
            Linear-9                    [16, 1]           1,025
          Sigmoid-10                    [16, 1]               0
           Linear-11                    [16, 3]           3,075
Total params: 537,012,676
Trainable params: 537,012,676
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 12.00

In [21]:
SHOW_IMG = False
if SHOW_IMG:
    def modeltorchviz(model, input_1, input_2):
        if input_2 != None:
            y = model(input_1.to(device), input_2.to(device))
        else:
            y = model(input_1to(device))
        if input_2 != None:
            MyConvNetVis = make_dot(y, params=dict(list(model.named_parameters()) + [('x', input_1)] + [('x', input_2)]))
        else:
            MyConvNetVis = make_dot(y, params=dict(list(model.named_parameters()) + [('x', input_1)]))
        MyConvNetVis.format = "png"
        MyConvNetVis.directory = "images"
        MyConvNetVis.view() 

In [22]:
if SHOW_IMG:
    modeltorchviz(generator, torch.randn(16, nz).requires_grad_(True), torch.randn(16, nb_label).requires_grad_(True))

In [23]:
if SHOW_IMG:
    modeltorchviz(discriminator, torch.randn(1, nc, ndf, ndf).requires_grad_(True))

In [24]:
def test(predict, labels):
    correct = 0
    predict = torch.argmax(predict, dim=1)
    labels = torch.argmax(labels, dim=1)
    correct = predict.eq(labels.data).cpu().sum()
    return correct, len(labels.data)

In [25]:
early_stop_count = 0
early_stop_patient = len(train_loader) * 20
early_stop = False
best_batch_loss = -1
save_model = False
print(early_stop_patient)

3780


In [26]:
def get_file_list(file_path, length):
    dir_list = os.listdir(file_path)
    if not dir_list:
        return
    else:
        dir_list = sorted(dir_list,  key=lambda x: os.path.getmtime(os.path.join(file_path, x)))
        print("files to be deleted ->", dir_list[0:length])
        return dir_list[0:length]

In [ ]:
for epoch in range(num_epochs):
    epoch += 1
    for i, data in enumerate(tqdm(train_loader, 0)):
        ###########################
        # (1) Update D network
        ###########################
        # train with real
        discriminator.zero_grad()
        img, label = data
        batch_size = img.size(0)
        with torch.no_grad():
            img, label = img.to(device), label.to(device)
            label = one_hot(label, num_classes=nb_label)
            s_label.resize_(batch_size).fill_(real_label)
            c_label.resize_(batch_size, 3).copy_(label)
        s_output, c_output = discriminator(img)
        s_errD_real = s_criterion(s_output[:,0], s_label.resize_(batch_size))
        c_errD_real = c_criterion(c_output, c_label)
        errD_real = s_errD_real + c_errD_real
        errD_real.backward()
        
        correct, length = test(c_output, c_label)

        # train with fake
        noise_data = np.random.normal(0, 1, (batch_size, nz))
        noise.resize_(batch_size, nz).data.copy_(torch.tensor(noise_data))
        label = np.random.randint(0, nb_label, batch_size)
        noise_ = np.random.normal(0, 1, (batch_size, nz))
        label_onehot = np.zeros((batch_size, nb_label))
        label_onehot[np.arange(batch_size), label] = 1
        c_label.resize_(batch_size, nb_label).data.copy_(torch.tensor(label_onehot))
        fake = generator(noise, c_label)
        s_label.data.fill_(fake_label)
        s_output,c_output = discriminator(fake)
        s_errD_fake = s_criterion(s_output[:,0], s_label.resize_(batch_size))
        c_errD_fake = c_criterion(c_output, c_label)
        errD_fake = s_errD_fake + c_errD_fake
        errD_fake.backward()

        errD = s_errD_real + s_errD_fake
        # errD.backward()
        optimizerD.step()

        ###########################
        # (2) Update G network
        ###########################
        generator.zero_grad()
        s_label.resize_(batch_size, 1).data.fill_(real_label)
        fake = generator(noise, c_label)
        s_output, c_output = discriminator(fake)
        s_errG = s_criterion(s_output[:,0], s_label.resize_(batch_size))
        c_errG = c_criterion(c_output, c_label)

        errG = s_errG + c_errG
        errG.backward()
        optimizerG.step()
        metrics['G_losses'].append(errG.item())
        metrics['G_class_losses'].append(c_errG.item())
        metrics['G_syn_losses'].append(s_errG.item())
        metrics['D_losses'].append(errD.item())
        metrics['D_real_losses'].append(errD_real.item())
        metrics['D_fake_losses'].append(errD_fake.item())
        metrics['D_class_losses'].append((c_errD_real + c_errD_fake).item())
        metrics['D_syn_losses'].append((s_errD_real + s_errD_fake).item())
        metrics['Losses'].append((errG + errD).item())
        metrics['Accuracy'].append(correct/length)

        print('[%d/%d][%d/%d] ========== Loss_D: %.4f, Loss_G: %.4f, Acc_G: %d/%d = %.4f'
              % (epoch, num_epochs, i + 1, len(train_loader),
                 errD.data, errG.data, correct, length, 100.0*correct/length))
    
        if best_batch_loss < 0:
            best_batch_loss = (errG + errD).item()
            early_stop_count = 0
        else:
            if best_batch_loss >= (errG + errD).item():
                best_batch_loss = (errG + errD).item()
                early_stop_count = 0
                print("---------------------< lowest loss update ->", best_batch_loss, ">---------------------")
                save_model = True
            else:
                early_stop_count += 1
                if early_stop_count >= early_stop_patient:
                    print("---------------------< early stopping ... >---------------------")
                    early_stop = True
                    break
        
        if i % round(len(train_loader)/10) == 0:
            vutils.save_image(img, './GANAug/output_images/ACGAN/real_samples_e' + str(epoch) + '_d' + str(i) + '.jpg', normalize=True)
            fake = generator(noise_fixed, label_fixed)
            for j in range(len(fake)):
                vutils.save_image(fake[j].data,
                        '%s/fake_samples_epoch_%03d.jpg' % ('./GANAug/output_images/ACGAN/' + label_dict[label_fixed_data[j]], epoch), 
                                  normalize=True)

    # do checkpointing
    if save_model == True:
        save_model = False
        torch.save(generator.state_dict(), '%s/G_epoch_%d_save_model.pth' % (os.path.join('.', 'GANAug/model/ACGAN'), epoch))
        torch.save(discriminator.state_dict(), '%s/D_epoch_%d_save_model.pth' % (os.path.join('.', 'GANAug/model/ACGAN'), epoch))
    elif early_stop:
        torch.save(generator.state_dict(), '%s/G_epoch_%d_early_stop.pth' % (os.path.join('.', 'GANAug/model/ACGAN'), epoch))
        torch.save(discriminator.state_dict(), '%s/D_epoch_%d_early_stop.pth' % (os.path.join('.', 'GANAug/model/ACGAN'), epoch))
        break
    elif epoch % round(num_epochs/10) == 0:
        save_model = False
        torch.save(generator.state_dict(), '%s/G_epoch_%d.pth' % (os.path.join('.', 'GANAug/model/ACGAN'), epoch))
        torch.save(discriminator.state_dict(), '%s/D_epoch_%d.pth' % (os.path.join('.', 'GANAug/model/ACGAN'), epoch))
    else:
        print("---------------------< no model saved at epoch:", epoch, ">---------------------")
    
    if len(os.listdir(os.path.join('.', 'GANAug/model/ACGAN'))) > 10:
        delete_list = get_file_list(os.path.join('.', 'GANAug/model/ACGAN'), len(os.listdir(os.path.join('.', 'GANAug/model/ACGAN'))) - 10)
        for file in delete_list:
            if os.path.exists(os.path.join(os.path.join('.', 'GANAug/model/ACGAN'), file)):
                os.remove(os.path.join(os.path.join('.', 'GANAug/model/ACGAN'), file))
            else:
                print("file ->", os.path.join(os.path.join('.', 'GANAug/model/ACGAN'), file), "does not exist")

  1%|▍                                                                                       | 1/189 [00:01<05:24,  1.72s/it]

[1/200][1/189] ========== Loss_D: 1.4541, Loss_G: 1.7165, Acc_G: 8/32 = 25.0000


  1%|▉                                                                                       | 2/189 [00:02<03:56,  1.26s/it]

[1/200][2/189] ========== Loss_D: 1.4410, Loss_G: 1.6653, Acc_G: 14/32 = 43.7500
---------------------< lowest loss update -> 3.106326103210449 >---------------------


  2%|█▍                                                                                      | 3/189 [00:04<04:11,  1.35s/it]

...

[6/200][68/189] ========== Loss_D: 1.4365, Loss_G: 1.7349, Acc_G: 24/32 = 75.0000


 37%|███████████████████████████████▊                                                       | 69/189 [01:29<02:37,  1.31s/it]

[6/200][69/189] ========== Loss_D: 1.4381, Loss_G: 1.5520, Acc_G: 23/32 = 71.8750


 37%|████████████████████████████████▏                                                      | 70/189 [01:30<02:31,  1.28s/it]

[6/200][70/189] ========== Loss_D: 1.4518, Loss_G: 1.6652, Acc_G: 23/32 = 71.8750


In [ ]:
def tr_plot(tr_data, start_epoch):
    Epoch_count=len(metrics['G_losses']) + start_epoch
    Epochs=[i + 1 for i in range (start_epoch ,Epoch_count)]  
    plt.figure(figsize=(20, 20))
    plt.style.use('fivethirtyeight')
    
    plt.subplot(2, 2, 1)
    plt.plot(Epochs, metrics['G_losses'], 'r', label='G_losses')
    plt.plot(Epochs, metrics['D_losses'], 'g',label='D_losses' )
    plt.title('G&D_losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    plt.plot(Epochs, metrics['G_class_losses'], 'r', label='G_class_losses')
    plt.plot(Epochs, metrics['G_syn_losses'], 'g',label='G_syn_losses' )
    plt.title('G_class&syn_losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 2, 3)
    plt.plot(Epochs, metrics['D_class_losses'], 'r', label='D_class_losses')
    plt.plot(Epochs, metrics['D_syn_losses'], 'g',label='D_syn_losses' )
    plt.title('D_class&syn_losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 2, 4)
    plt.plot(Epochs, metrics['D_real_losses'], 'r', label='D_real_losses')
    plt.plot(Epochs, metrics['D_fake_losses'], 'g',label='D_fake_losses' )
    plt.title('D_real&fake_losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout
    plt.show()

In [ ]:
tr_plot(metrics, 0)

In [ ]:
def tr_plot(tr_data, start_epoch):
    Epoch_count=len(metrics['Losses']) + start_epoch
    Epochs=[i + 1 for i in range (start_epoch ,Epoch_count)]  
    plt.figure(figsize=(20, 10))
    plt.style.use('fivethirtyeight')
    
    plt.subplot(2, 2, 1)
    plt.plot(Epochs, metrics['Losses'], 'blue', label='Loss')
    plt.title('Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    plt.plot(Epochs, metrics['Accuracy'], 'orange', label='Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout
    plt.show()

In [ ]:
tr_plot(metrics, 0)

In [ ]:
def test_gan(generator, discriminator, num_epochs, metrics, loader):
    now = datetime.datetime.now()
    g_losses = metrics['G_losses'][-1]
    d_losses = metrics['D_losses'][-1]
    path='GANAug/output_images/ACGAN'
    try:
        os.mkdir(os.path.join('.', path))
    except Exception as error:
        print(error)

    real_batch = next(iter(loader))
    
    test_img_list = []
    test_noise = torch.randn(batch_size, nz, device=device)
    test_label = torch.randn(batch_size, nb_label, device=device)
    test_fake = generator(test_noise, test_label).detach().cpu()
    test_img_list.append(vutils.make_grid(test_fake, padding=2, normalize=True))

    fig = plt.figure(figsize=(20, 20))
    ax1 = plt.subplot(1,2,1)
    ax1 = plt.axis("off")
    ax1 = plt.title("Real Images")
    ax1 = plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

    ax2 = plt.subplot(1,2,2)
    ax2 = plt.axis("off")
    ax2 = plt.title("Fake Images")
    ax2 = plt.imshow(np.transpose(test_img_list[-1],(1,2,0)))
    plt.show()
    fig.savefig('%s/image_%.3f_%.3f_%d_%s.png' %
                   (path, g_losses, d_losses, num_epochs, now.strftime("%Y-%m-%d_%H:%M:%S")))

In [ ]:
test_gan(generator, discriminator, num_epochs, metrics, train_loader)

In [ ]:
def plot_gan(name, train_epoch, values, path, save):
    clear_output(wait=True)
    plt.close('all')
    fig = plt.figure()
    fig = plt.ion()
    fig = plt.subplot(1, 1, 1)
    fig = plt.title('epoch: %s -> %s: %s' % (train_epoch, name, values[-1]))
    fig = plt.ylabel(name)
    fig = plt.xlabel('train_set')
    fig = plt.plot(values)
    fig = plt.grid()
    get_fig = plt.gcf()
    fig = plt.draw()  # draw the plot
    fig = plt.pause(1)  # show it for 1 second
    plt.show()
    if save:
        now = datetime.datetime.now()
        get_fig.savefig('%s/%s_%.3f_%d_%s.png' %
                        (path, name, train_epoch, values[-1], now.strftime("%Y-%m-%d_%H:%M:%S")))

In [ ]:
def save_model(generator, discriminator, gen_optimizer, dis_optimizer, metrics, num_epochs):
    now = datetime.datetime.now()
    g_losses = metrics['G_losses'][-1]
    d_losses = metrics['D_losses'][-1]
    path='GANAug/plots/ACGAN/train_%+.3f_%+.3f_%s'% (g_losses, d_losses, now.strftime("%Y-%m-%d_%H-%M-%S"))
    try:
        os.mkdir(os.path.join('.', path))
    except Exception as error:
        print(error)
    plot_gan('G_losses', num_epochs, metrics['G_losses'], path, True)
    plot_gan('D_losses', num_epochs, metrics['D_losses'], path, True)
    plot_gan('G_syn_losses', num_epochs, metrics['G_syn_losses'], path, True)
    plot_gan('G_class_losses', num_epochs, metrics['G_class_losses'], path, True)
    plot_gan('G_syn_losses', num_epochs, metrics['G_syn_losses'], path, True)
    plot_gan('D_class_losses', num_epochs, metrics['D_class_losses'], path, True)
    plot_gan('D_syn_losses', num_epochs, metrics['D_syn_losses'], path, True)
    plot_gan('Losses', num_epochs, metrics['Losses'], path, True)
    plot_gan('Accuracy', num_epochs, metrics['Accuracy'], path, True)

In [ ]:
save_model(generator, discriminator, optimizerG, optimizerD, metrics, num_epochs)

In [ ]:
test_img_list = []
test_noise = torch.randn(batch_size, nz, device=device)
test_label = torch.randn(batch_size, nb_label, device=device)
test_img = generator(test_noise, test_label)

s_output, c_label_op = discriminator(test_img.detach().to(device))
print('Discriminator s', s_output)
print('Discriminator c', c_label_op)

test_img = test_img.detach().cpu()
test_img_list.append(vutils.make_grid(test_img, padding=2, normalize=True))
plt.imshow(np.transpose(test_img_list[-1],(1,2,0)))

In [ ]:
data = next(iter(test_loader))
test_noise, test_class_lable = data
test_img = test_noise
print('class label for real', test_class_lable)

s_output,c_label_op = discriminator(test_img.detach().to(device))
print('Discriminator s', s_output)
print('Discriminator c', c_label_op)

test_img = test_img.detach().cpu()
test_img_list.append(vutils.make_grid(test_img, padding=2, normalize=True))
plt.imshow(np.transpose(test_img_list[-1],(1,2,0)))